In [8]:
import networkx as nx
from pyvis.network import Network
import itertools

In [9]:
# --- STEP 1: MANUAL DATA ENTRY ---
# Define your papers as a list of lists.
# Each inner list represents one paper/project and contains the names of the authors.
# Ensure you spell names consistently (e.g., always use "Collaborator A", not "Collab A").

papers = [
    ["ribeiro_vh", "chinellato_dd", "lisa_ma", "serenone_wm", "shen_c", "takahashi_j","torrieri_g"],
    ["ribeiro_vh", "caligiuri_a", "emery_e", "ferreira_l", "garcia-castillo_j", "lindner_sd", "molina-hernandez_j", "passos_nara", "sartore_m", "wang_b", "calleja-solanas_v"],
    ["ribeiro_vh", "sunahara_as", "shahtahmassebi_g", "perc_m", "ribeiro_hv"],
    # Add more papers here...
]

In [10]:
# --- STEP 2: BUILD THE NETWORK ---
G = nx.Graph()

for paper_authors in papers:
    # 1. Add authors as nodes
    for author in paper_authors:
        if not G.has_node(author):
            G.add_node(author, title=author, group=1) # 'title' shows on hover
    
    # 2. Create edges (connections)
    # itertools.combinations creates all possible pairs of authors in the paper
    # e.g., [A, B, C] becomes (A,B), (A,C), (B,C)
    for source, target in itertools.combinations(paper_authors, 2):
        if G.has_edge(source, target):
            # If they collaborated before, increase the weight (thickness of line)
            G[source][target]['weight'] += 1
            G[source][target]['value'] += 1 # 'value' controls visual thickness in PyVis
            G[source][target]['title'] = f"Collaborations: {G[source][target]['weight']}"
        else:
            # New collaboration
            G.add_edge(source, target, weight=1, value=1, title="Collaborations: 1")

In [11]:
# --- STEP 3: CALCULATE CENTRALITY & SIZE NODES ---
# We use simple Degree (number of connections) for size.
# You could also use nx.betweenness_centrality(G) if you want to highlight "bridges".

degrees = dict(G.degree)

# We iterate through nodes to assign attributes
for node in G.nodes():
    # BASE SIZE: 
    # PyVis uses 'value' to scale nodes. 
    # We use the degree count directly. 
    degree_value = degrees[node]
    
    # NODE COLOR:
    # Let's make YOU distinct from the others.
    if node == "Vítor Hugo Ribeiro":
        color = "#ff5733" # A distinct orange/red for you
        size_multiplier = 1.5 # Make yourself slightly larger than your raw degree suggests
    else:
        color = "#97c2fc" # Standard PyVis blue
        size_multiplier = 1.0

    # Update the node attributes
    G.nodes[node]['value'] = degree_value * size_multiplier
    G.nodes[node]['color'] = color
    G.nodes[node]['title'] = f"{node}<br>Collaborators: {degree_value}" # Tooltip on hover
    G.nodes[node]['label'] = node # Show name always

In [12]:
# --- STEP 4: VISUALIZATION SETTINGS ---
net = Network(height="650px", width="100%", bgcolor="#ffffff", font_color="black")
net.from_nx(G)

# Physics settings for a nice spread
net.repulsion(
    node_distance=200, 
    central_gravity=0.2, 
    spring_length=150, 
    spring_strength=0.05, 
    damping=0.09
)

In [15]:
# --- STEP 5: EXPORT ---
output_filename = "/home/vitor/Dropbox/mypage/vitorhribeiro.github.io/static/interactive/collaboration_network.html"
net.write_html(output_filename)
print(f"Success! Network generated with sized nodes: {output_filename}")

Success! Network generated with sized nodes: /home/vitor/Dropbox/mypage/vitorhribeiro.github.io/static/interactive/collaboration_network.html
